In [1]:
import geopandas as gpd
import pandas as pd

# Load data

In [2]:
parcels = gpd.read_file('la_parcels/Parcels.shp')

In [3]:
parcels.shape

(881522, 18)

In [4]:
sites_df = pd.read_excel('Appendix 4.7 Candidate Sites For Rezoning-Table B.xlsx', skiprows=1)

In [5]:
sites_df['PIN'].isin(parcels['PIN']).mean()

0.9990280410166691

In [6]:
(~sites_df['PIN'].isin(parcels['PIN'])).sum()

260

# Merge data

In [7]:
merged_df = parcels[['PIN', 'geometry']].merge(
    sites_df,
    how='right',
    on='PIN'
)

# Save merged data

In [10]:
merged_df_crs = merged_df.to_crs('EPSG:4326')

merged_df_crs['units'] = (
    pd.to_numeric(merged_df_crs['Maximum Density Allowed'], errors='coerce')
    * merged_df_crs['Parcel Size\n(Acres)']
).round(1)

merged_df_crs['density'] = pd.to_numeric(
    merged_df_crs['Maximum Density Allowed'],
    errors='coerce'
)

merged_df_crs['zoning'] = merged_df_crs['Current Zoning']

In [11]:
merged_df_crs.geometry.total_bounds

array([-118.66755417,   33.7064461 , -118.1555711 ,   34.32457583])

In [12]:
merged_df_crs[[
    'geometry', 'zoning', 'units', 'density'
]].dropna(subset=['geometry']).to_file('out.geojson', driver='GeoJSON')

In [13]:
points_df = merged_df_crs.copy()
points_df.geometry = points_df.geometry.centroid

points_df[[
    'geometry', 'zoning', 'units', 'density'
]].dropna(subset=['geometry']).to_file('points.geojson', driver='GeoJSON')

/var/folders/y6/q_9gyvlj7ln20t8tgvcpmfcr0000gn/T/ipykernel_3675/3972787880.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  points_df.geometry = points_df.geometry.centroid
